Use fancy machine learning to predict whether an article makes it into Nature/Science or PRL.

In [1]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
import matplotlib.pyplot as plt 
import time
import numpy as np
import scipy as sp

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
from nltk.corpus import stopwords
s=stopwords.words('english') + ['']

# Machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.externals import joblib

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [2]:
# Save model
clf = joblib.load('text_abstract.pkl')

Latest issue of PRL is Volume 116, Issue 4: http://journals.aps.org/prl/issues/116/4. Below are the highlight articles.

In [3]:
abstractsPRL = ['We present significant evidence of halo assembly bias for SDSS redMaPPer galaxy clusters in the redshift range [0.1, 0.33]. By dividing the 8,648 clusters into two subsamples based on the average member galaxy separation from the cluster center, we first show that the two subsamples have very similar halo mass of M200m≃1.9×1014  h−1M⊙ based on the weak lensing signals at small radii R≲10  h−1Mpc. However, their halo bias inferred from both the large-scale weak lensing and the projected autocorrelation functions differs by a factor of ∼1.5, which is a signature of assembly bias. The same bias hypothesis for the two subsamples is excluded at 2.5σ in the weak lensing and 4.4σ in the autocorrelation data, respectively. This result could bring a significant impact on both galaxy evolution and precision cosmology.',
                'Astrophysical observations spanning dwarf galaxies to galaxy clusters indicate that dark matter (DM) halos are less dense in their central regions compared to expectations from collisionless DM N-body simulations. Using detailed fits to DM halos of galaxies and clusters, we show that self-interacting DM (SIDM) may provide a consistent solution to the DM deficit problem across all scales, even though individual systems exhibit a wide diversity in halo properties. Since the characteristic velocity of DM particles varies across these systems, we are able to measure the self-interaction cross section as a function of kinetic energy and thereby deduce the SIDM particle physics model parameters. Our results prefer a mildly velocity-dependent cross section, from σ/m≈2  cm2/g on galaxy scales to σ/m≈0.1  cm2/g on cluster scales, consistent with the upper limits from merging clusters. Our results dramatically improve the constraints on SIDM models and may allow the masses of both DM and dark mediator particles to be measured even if the dark sector is completely hidden from the standard model, which we illustrate for the dark photon model.',
                'We report the observation of efficient and low-noise frequency conversion between two microwave modes, mediated by the motion of a mechanical resonator subjected to radiation pressure. We achieve coherent conversion of more than 1012  photons/s with a 95% efficiency and a 14 kHz bandwidth. With less than 10−1  photons·s−1·Hz−1 of added noise, this optomechanical frequency converter is suitable for quantum state transduction. We show the ability to operate this converter as a tunable beam splitter, with direct applications for photon routing and communication through complex quantum networks.',
                'Thermodynamic properties of matter are conveniently expressed as functional relations between variables known as equations of state. Here we experimentally determine the compressibility, density, and pressure equations of state for an attractive 2D Fermi gas in the normal phase as a function of temperature and interaction strength. In 2D, interacting gases exhibit qualitatively different features to those found in 3D. This is evident in the normalized density equation of state, which peaks at intermediate densities corresponding to the crossover from classical to quantum behavior.',
                'We report the experimental measurement of the equation of state of a two-dimensional Fermi gas with attractive s-wave interactions throughout the crossover from a weakly coupled Fermi gas to a Bose gas of tightly bound dimers as the interaction strength is varied. We demonstrate that interactions lead to a renormalization of the density of the Fermi gas by several orders of magnitude. We compare our data near the ground state and at finite temperature with predictions for both fermions and bosons from quantum Monte Carlo simulations and Luttinger-Ward theory. Our results serve as input for investigations of close-to-equilibrium dynamics and transport in the two-dimensional system.',
                'The Wigner function provides a useful quasiprobability representation of quantum mechanics, with applications in various branches of physics. Many nice properties of the Wigner function are intimately connected with the high symmetry of the underlying operator basis composed of phase point operators: any pair of phase point operators can be transformed to any other pair by a unitary symmetry transformation. We prove that, in the discrete scenario, this permutation symmetry is equivalent to the symmetry group being a unitary 2 design. Such a highly symmetric representation can only appear in odd prime power dimensions besides dimensions 2 and 8. It suffices to single out a unique discrete Wigner function among all possible quasiprobability representations. In the course of our study, we show that this discrete Wigner function is uniquely determined by Clifford covariance, while no Wigner function is Clifford covariant in any even prime power dimension.',
                'We derive the first ever on-shell recursion relations applicable to effective field theories. Based solely on factorization and the soft behavior of amplitudes, these recursion relations employ a new rescaling momentum shift to construct all tree-level scattering amplitudes in the nonlinear sigma model, Dirac-Born-Infeld theory, and the Galileon. Our results prove that all theories with enhanced soft behavior are on-shell constructible.',
                'We use time-dependent density functional theory to study self-irradiated Si. We calculate the electronic stopping power of Si in Si by evaluating the energy transferred to the electrons per unit path length by an ion of kinetic energy from 1 eV to 100 keV moving through the host. Electronic stopping is found to be significant below the threshold velocity normally identified with transitions across the band gap. A structured crossover at low velocity exists in place of a hard threshold. An analysis of the time dependence of the transition rates using coupled linear rate equations enables one of the excitation mechanisms to be clearly identified: a defect state induced in the gap by the moving ion acts like an elevator and carries electrons across the band gap.',
                'We demonstrate coherent population trapping of a single nuclear spin in a room-temperature solid. To this end, we exploit a three-level system with a Λ configuration in the microwave domain, which consists of nuclear spin states addressed through their hyperfine coupling to the electron spin of a single nitrogen-vacancy defect in diamond. Moreover, the Λ-scheme relaxation is externally controlled through incoherent optical pumping and separated in time from consecutive coherent microwave excitations. Such a scheme allows us (i) to monitor the sequential accumulation of population into the dark state and (ii) to reach a novel regime of coherent population trapping dynamics for which periodic arrays of dark resonances can be observed, owing to multiple constructive interferences. This Letter offers new prospects for quantum state preparation, information storage in hybrid quantum systems, and metrology.',
                'We extract the phase coherence of a qubit defined by singlet and triplet electronic states in a gated GaAs triple quantum dot, measuring on time scales much shorter than the decorrelation time of the environmental noise. In this nonergodic regime, we observe that the coherence is boosted and several dephasing times emerge, depending on how the phase stability is extracted. We elucidate their mutual relations, and demonstrate that they reflect the noise short-time dynamics.',
                'By systematically varying the mobility of self-propelled particles in a 2D lattice, we experimentally study the influence of particle mobility on system’s collective motion. Our system is intrinsically nonequilibrium due to the lack of energy equipartition. By constructing the covariance matrix of spatial fluctuations and solving for its eigenmodes, we obtain the collective motions of the system with various magnitudes. Interestingly, our structurally ordered nonequilibrium system exhibits properties almost identical to disordered glassy systems under thermal equilibrium: the modes with large overall motions are spatially correlated and quasilocalized while the modes with small collective motions are highly localized, resembling the low- and high-frequency modes in glass. More surprisingly, a peak similar to the boson peak forms in our nonequilibrium system as the number of mobile particles increases, revealing the possible origin of the boson peak from a dynamic aspect. We further illustrate that the spatially correlated large-movement modes can be produced by the cooperation of highly active particles above a threshold fraction, while the localized small-movement modes can be created by adding individual inactive particles. Our study clarifies the role of mobility in collective motions, and further suggests a promising possibility of extending the powerful mode analysis approach to nonequilibrium systems.',
               ]

Latest issue of Nature is Volume 530 Number 7588 pp6-124

In [4]:
abstractsNature = ['Cometary nuclei consist mostly of dust and water ice1. Previous observations have found nuclei to be low-density and highly porous bodies2, 3, 4, but have only moderately constrained the range of allowed densities because of the measurement uncertainties. Here we report the precise mass, bulk density, porosity and internal structure of the nucleus of comet 67P/Churyumov–Gerasimenko on the basis of its gravity field. The mass and gravity field are derived from measured spacecraft velocity perturbations at fly-by distances between 10 and 100 kilometres. The gravitational point mass is GM = 666.2 ± 0.2 cubic metres per second squared, giving a mass M = (9,982 ± 3) × 109 kilograms. Together with the current estimate of the volume of the nucleus5, the average bulk density of the nucleus is 533 ± 6 kilograms per cubic metre. The nucleus appears to be a low-density, highly porous (72–74 per cent) dusty body, similar to that of comet 9P/Tempel 12, 3. The most likely composition mix has approximately four times more dust than ice by mass and two times more dust than ice by volume. We conclude that the interior of the nucleus is homogeneous and constant in density on a global scale without large voids. The high porosity seems to be an inherent property of the nucleus material.',
                   'The time it takes a bound electron to respond to the electromagnetic force of light sets a fundamental speed limit on the dynamic control of matter and electromagnetic signal processing. Time-integrated measurements of the nonlinear refractive index1 of matter indicate that the nonlinear response of bound electrons to optical fields is not instantaneous; however, a complete spectral characterization of the nonlinear susceptibility tensors2—which is essential to deduce the temporal response of a medium to arbitrary driving forces using spectral measurements—has not yet been achieved. With the establishment of attosecond chronoscopy3, 4, 5, the impulsive response of positive-energy electrons to electromagnetic fields has been explored through ionization of atoms6 and solids7 by an extreme-ultraviolet attosecond pulse8 or by strong near-infrared fields9, 10, 11. However, none of the attosecond studies carried out so far have provided direct access to the nonlinear response of bound electrons. Here we demonstrate that intense optical attosecond pulses synthesized in the visible and nearby spectral ranges allow sub-femtosecond control and metrology of bound-electron dynamics. Vacuum ultraviolet spectra emanating from krypton atoms, exposed to intense waveform-controlled optical attosecond pulses, reveal a finite nonlinear response time of bound electrons of up to 115 attoseconds, which is sensitive to and controllable by the super-octave optical field. Our study could enable new spectroscopies of bound electrons in atomic, molecular or lattice potentials of solids12, as well as light-based electronics operating on sub-femtosecond timescales and at petahertz rates13, 14, 15.',
                   'Many procedures in modern clinical medicine rely on the use of electronic implants in treating conditions that range from acute coronary events to traumatic injury1, 2. However, standard permanent electronic hardware acts as a nidus for infection: bacteria form biofilms along percutaneous wires, or seed haematogenously, with the potential to migrate within the body and to provoke immune-mediated pathological tissue reactions3, 4. The associated surgical retrieval procedures, meanwhile, subject patients to the distress associated with re-operation and expose them to additional complications5, 6, 7, 8. Here, we report materials, device architectures, integration strategies, and in vivo demonstrations in rats of implantable, multifunctional silicon sensors for the brain, for which all of the constituent materials naturally resorb via hydrolysis and/or metabolic action9, 10, 11, 12, eliminating the need for extraction. Continuous monitoring of intracranial pressure and temperature illustrates functionality essential to the treatment of traumatic brain injury2, 13; the measurement performance of our resorbable devices compares favourably with that of non-resorbable clinical standards. In our experiments, insulated percutaneous wires connect to an externally mounted, miniaturized wireless potentiostat for data transmission. In a separate set-up, we connect a sensor to an implanted (but only partially resorbable) data-communication system, proving the principle that there is no need for any percutaneous wiring. The devices can be adapted to sense fluid flow, motion, pH or thermal characteristics, in formats that are compatible with the body’s abdomen and extremities, as well as the deep brain, suggesting that the sensors might meet many needs in clinical medicine.',
                   'The splitting of water photoelectrochemically into hydrogen and oxygen represents a promising technology for converting solar energy to fuel1, 2. The main challenge is to ensure that photogenerated holes efficiently oxidize water, which generally requires modification of the photoanode with an oxygen evolution catalyst (OEC) to increase the photocurrent and reduce the onset potential3. However, because excess OEC material can hinder light absorption and decrease photoanode performance4, its deposition needs to be carefully controlled—yet it is unclear which semiconductor surface sites give optimal improvement if targeted for OEC deposition, and whether sites catalysing water oxidation also contribute to competing charge-carrier recombination with photogenerated electrons5. Surface heterogeneity6 exacerbates these uncertainties, especially for nanostructured photoanodes benefiting from small charge-carrier transport distances1, 7, 8. Here we use super-resolution imaging9, 10, 11, 12, 13, operated in a charge-carrier-selective manner and with a spatiotemporal resolution of approximately 30 nanometres and 15 milliseconds, to map both the electron- and hole-driven photoelectrocatalytic activities on single titanium oxide nanorods. We then map, with sub-particle resolution (about 390 nanometres), the photocurrent associated with water oxidation, and find that the most active sites for water oxidation are also the most important sites for charge-carrier recombination. Site-selective deposition of an OEC, guided by the activity maps, improves the overall performance of a given nanorod—even though more improvement in photocurrent efficiency correlates with less reduction in onset potential (and vice versa) at the sub-particle level. Moreover, the optimal catalyst deposition sites for photocurrent enhancement are the lower-activity sites, and for onset potential reduction the optimal sites are the sites with more positive onset potential, contrary to what is obtainable under typical deposition conditions. These findings allow us to suggest an activity-based strategy for rationally engineering catalyst-improved photoelectrodes, which should be widely applicable because our measurements can be performed for many different semiconductor and catalyst materials.',
                   ]

In [11]:
abstractsNature2 = [x.lower().replace('here', '') for x in abstractsNature]

In [13]:
predict = clf.predict(abstractsPRL + abstractsNature2)
target = [0]*len(abstractsPRL) + [1]*len(abstractsNature2)
print predict
print target

[0 0 1 0 0 0 0 0 0 0 0 1 1 1 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [14]:
#SVC(kernel='linear')
print(metrics.classification_report(target, predict,
                                    target_names=['PRL', 'Nature/Science']))
print('Accuracy score: %0.2f' % metrics.accuracy_score(target, predict))

                precision    recall  f1-score   support

           PRL       1.00      0.91      0.95        11
Nature/Science       0.80      1.00      0.89         4

   avg / total       0.95      0.93      0.94        15

Accuracy score: 0.93


In [16]:
print clf.predict(['we report quantum information star'])
print clf.predict(['We consider condensate tensor squeezing'])

[1]
[0]
